## 13.1 通过重定向/管道/文件接受输入

## 13.3 解析命令行选项

argparse 模块可被用来解析命令行选项。

argparse 模块是标准库中最大的模块之一,拥有大量的配置选项。本节只是演示
了其中最基础的一些特性,帮助你入门。
为 了 解 析 命 令 行 选 项, 你 首 先 要 创 建 一 个 ArgumentParser 实 例,并 使 用
add_argument() 方法声明你想要支持的选项。在每个 add_argument() 调用中,dest
参数指定解析结果被指派给属性的名字。metavar 参数被用来生成帮助信息。action
参数指定跟属性对应的处理逻辑,通常的值为 store , 被用来存储某个值或讲多个参数
值收集到一个列表中。

## 13.6 执行外部命令并获取它的输出

你想执行一个外部命令并以 Python 字符串的形式获取执行结果。使用 `subprocess.check_output()` 函数

In [ ]:
out_bytes = subprocess.check_output(['cmd','arg1','arg2'],
stderr=subprocess.STDOUT)

通常来讲,命令的执行不需要使用到底层 shell 环境(比如 sh、bash)。一个字符串
列表会被传递给一个低级系统命令,比如 os.execve() 。

如果你想让命令被一个 shell
执行,传递一个字符串参数,并设置参数 shell=True . 有时候你想要 Python 去执行
一个复杂的 shell 命令的时候这个就很有用了,比如管道流、I/O 重定向和其他特性。
例如:

In [ ]:
out_bytes = subprocess.check_output('grep python | wc > out', shell=True)

使用 check_output() 函数是执行外部命令并获取其返回值的最简单方式。

如果你需要对子进程做更复杂的交互,比如给它发送输入,你得采用另外一种方法。这
时候可直接使用 `subprocess.Popen` 类。

In [ ]:
# Launch a command with pipes
p = subprocess.Popen(['wc'],
stdout = subprocess.PIPE,
stdin = subprocess.PIPE)
# Send the data and get the output
stdout, stderr = p.communicate(text)
# To interpret as text, decode
out = stdout.decode('utf-8')
err = stderr.decode('utf-8')

**subprocess 模块对于依赖 TTY 的外部命令不合适用。**例如,你不能使用它来自
动化一个用户输入密码的任务(比如一个 ssh 会话)。这时候,你需要使用到第三方模
块了,比如基于著名的 expect 家族的工具(pexpect 或类似的)

## 13.7 复制或者移动文件和目录

`shutil 模块`有很多便捷的函数可以复制文件和目录。

import shutil
# Copy src to dst. (cp src dst)
shutil.copy(src, dst)
# Copy files, but preserve metadata (cp -p src dst)
shutil.copy2(src, dst)
# Copy directory tree (cp -R src dst)
shutil.copytree(src, dst)
# Move src to dst (mv src dst)
shutil.move(src, dst)

`copytree()` 可以让你在复制过程中选择性的`忽略某些文件或目录`。你可以提供一
个忽略函数,接受一个目录名和文件名列表作为输入,`返回一个忽略的名称列表`。

In [ ]:
def ignore_pyc_files(dirname, filenames):
    return [name in filenames if name.endswith('.pyc')]

shutil.copytree(src, dst, ignore=ignore_pyc_files)

当`处理文件名`的时候,最好使用 `os.path`
中的函数来确保最大的可移植性(特别是同时要适用于 Unix 和 Windows)。

In [ ]:
>>> filename = '/Users/guido/programs/spam.py'
>>> import os.path
>>> os.path.basename(filename)
'spam.py'
>>> os.path.dirname(filename)
'/Users/guido/programs'
>>> os.path.split(filename)
('/Users/guido/programs', 'spam.py')
>>> os.path.join('/new/dir', os.path.basename(filename))
'/new/dir/spam.py'
>>> os.path.expanduser('~/guido/programs/spam.py')
'/Users/guido/programs/spam.py'
>>>

## 13.8 创建和解压归档文件

`shutil 模块`拥有两个函数——`make_archive()` 和 `unpack_archive()` 可派上用
场。

In [ ]:
>>> import shutil
>>> shutil.unpack_archive('Python-3.3.0.tgz')
>>> shutil.make_archive('py33','zip','Python-3.3.0')
'/Users/beazley/Downloads/py33.zip'
>>>

make_archive() 的 第 二 个 参 数 是 期 望 的 输 出 格 式。可 以 使 用`get_archive_formats()` 获取所有支持的`归档格式`列表。

## 13.9 通过文件名查找文件

查找文件,可使用 `os.walk()` 函数,传一个顶级目录名给它。

os.walk() 方法为我们遍历目录树,每次进入一个目录,它会返回一个三元组,包
含相对于查找目录的相对路径,一个该目录下的目录名列表,以及那个目录下面的文件
名列表。

In [ ]:
#!/usr/bin/env python
import os

def findfile(start, name):
    for relpath, dirs, files in os.walk(start):
        if name in files:
            full_path = os.path.join(start, relpath, name)
            print(os.path.normpath(os.path.abspath(full_path)))

if __name__ == '__main__':
    findfile(sys.argv[1], sys.argv[2])

打印
所有最近被修改过的文件:

In [ ]:
#!/usr/bin/env python3.3
import time
import os

def modified_within(top, seconds):
    now = time.time()
    for path, dirs, files in os.walk(top):
        for name in files:
            fullpath = os.path.join(path, name)
            if os.path.exists(fullpath):
                mtime = os.path.getmtime(fullpath)
                if mtime > (now - seconds):
                    print(fullpath)
                    

if __name__ == '__main__':
    import sys
    if len(sys.argv) != 3:
        print('Usage: {} dir seconds'.format(sys.argv[0]))
        raise SystemExit(1)
    modified_within(sys.argv[1], float(sys.argv[2]))

## 13.10 读取配置文件

读取普通.ini 格式的配置文件, `configparser 模块`

In [ ]:
>>> from configparser import ConfigParser
>>> cfg = ConfigParser()
>>> cfg.read('config.ini')
['config.ini']
>>> cfg.sections()
['installation', 'debug', 'server']
>>> cfg.get('installation','library')
'/usr/local/lib'
>>> cfg.getboolean('debug','log_errors')
True
>>> cfg.getint('server','port')
8080

In [ ]:
>>> cfg.set('server','port','9000')
>>> cfg.set('debug','log_errors','False')
>>> import sys
>>> cfg.write(sys.stdout)

配置文件中的**配置项名字是不区分大小写的**。例如:

In [ ]:
>>> cfg.get('installation','PREFIX')
'/usr/local'
>>> cfg.get('installation','prefix')
'/usr/local'
>>>

getboolean() 方法查找任何可行的值。例如下面都是等价的:

In [ ]:
log_errors = true
log_errors = TRUE
log_errors = Yes
log_errors = 1

或许配置文件和 Python 代码最大的不同在于,它并不是从上而下的顺序执行。文
件是安装一个整体被读取的。如果碰到了变量替换,它实际上已经被替换完成了。

例
如,在下面这个配置中,prefix 变量在使用它的变量之前或之后定义都是可以的:

ConfigParser 有个容易被忽视的特性是**它能一次读取多个配置文件然后合并成一
个配置。** 能跟之前的配置合并起来。

In [ ]:
>>> # Previously read configuration
>>> cfg.get('installation', 'prefix')
'/usr/local'

>>> # Merge in user-specific configuration
>>> import os
>>> cfg.read(os.path.expanduser('~/.config.ini'))
['/Users/beazley/.config.ini']
>>> cfg.get('installation', 'prefix')
'/Users/beazley/test'
>>> cfg.get('installation', 'library')
'/Users/beazley/test/lib'
>>> cfg.getboolean('debug', 'log_errors')
False
>>>

## 13.11 给简单脚本增加日志功能

## 13.12 给函数库增加日志功能

调用 getLogger(__name__) 创建一个和调用模块同名的 logger 模块。由于模块都是唯一的,因此创建的 logger 也将是唯一的。

In [ ]:
logging.getLogger('somelib').level=logging.DEBUG